In [181]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
import pandas as pd
import json
import math

Conexão com a plataforma Azure Blob Storage

In [182]:
#enter credentials
account_name = "filmesteste"
account_access_key = "S5wta6QAtuV6yYubHeMSen4mO9CHuP3A0kIYyZmEpEWj6uUugxXzYdrpMuVGxn6Td9eQU+gmHRfw+AStpgojeQ=="
container_name = "filmesbd"


In [183]:
#create a client to interact with blob storage
connect_str = 'DefaultEndpointsProtocol=https;AccountName=' + account_name + ';AccountKey=' + account_access_key + ';EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [184]:
#use the client to connect to the container
container_client = blob_service_client.get_container_client(container_name)

In [185]:
def get_data(file_name):
    blob_client = container_client.get_blob_client(file_name)
    stream_downloader = blob_client.download_blob()
    stream = json.loads(stream_downloader.readall())
    return pd.DataFrame(stream)

In [186]:

blob_list = []
for blob_i in container_client.list_blobs():
    blob_list.append(blob_i.name)
print(blob_list)

['actors.json', 'directors.json', 'movies.json', 'producers.json', 'ratings.json', 'tmdb_5000_movies.json']


EXTRAÇÃO DE DADOS

In [187]:
bases = {}
for nome_arquivo in blob_list:
    bases[nome_arquivo] = get_data(nome_arquivo)

In [188]:
#redefinindo nome da base de dados utilizadas no trabalho e definidas no catálogo de dados
df_movies = bases['movies.json']
df_ratings = bases['ratings.json']
df_box_office = bases['tmdb_5000_movies.json']
#_*_*_**_***_*_**_**_*_***_*_*
df_actors = bases['actors.json']
df_directors = bases['directors.json']
df_producers = bases['producers.json']

TRANSFORMAÇÃO DE DADOS

In [189]:
n_movies_without_actors = len(df_movies[df_movies.all_actors.isna()])
n_movies_without_directors = len(df_movies[df_movies.director.isna()])
n_movies_without_producers = len(df_movies[df_movies.producer.isna()])

print('FILMES SEM LISTA DE ATORES: ' + str(n_movies_without_actors))
print('FILMES SEM LISTA DE DIRETORES: ' + str(n_movies_without_directors))
print('FILMES SEM LISTA DE PRODUTORES: ' + str(n_movies_without_producers))

FILMES SEM LISTA DE ATORES: 321
FILMES SEM LISTA DE DIRETORES: 267
FILMES SEM LISTA DE PRODUTORES: 930


In [190]:
#REMOVE TODOS OS FILMES OS QUAIS NÃO POSSUEM OS ATORES, DIRETORES E PRODUTORES QUE TRABAHARAM NELES.

df_movies = df_movies[df_movies.all_actors.notna()]
df_movies = df_movies[df_movies.director.notna()]
df_movies = df_movies[df_movies.producer.notna()]

In [191]:
#Redefina o índice do DataFrame e use o padrão.
#inplace = True : Se deve modificar o DataFrame em vez de criar um novo.
#drop = True :This resets the index to the default integer index.
df_movies.reset_index(inplace=True, drop=True)

In [192]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14238 entries, 0 to 14237
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            14238 non-null  object 
 1   year             14238 non-null  int64  
 2   genre            14238 non-null  object 
 3   _id              14238 non-null  object 
 4   director         14238 non-null  object 
 5   all_actors       14238 non-null  object 
 6   gender_percent   14182 non-null  float64
 7   adjusted_budget  5435 non-null   float64
 8   producer         14238 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1001.2+ KB


CONTABILIZANDO A QUANTIDADE DE ATORES E ATRIZES

In [193]:
df_movies_final = pd.DataFrame()
for index, movie in df_movies.iterrows(): #Iterar sobre linhas do DataFrame como pares (índice, série).
    male_count = 0
    female_count = 0
    for actor in movie.all_actors:
        
        if actor['gender'] == 'male':
            male_count += 1
        else:
            female_count +=1
            
    movie['actors_count'] = male_count
    movie['actresses_count'] = female_count
    
    df_movies_final = pd.concat([df_movies_final, pd.DataFrame([movie])], ignore_index=True)

CONTABILIZANDO A QUANTIDADE DE PRODUTORES HOMENS E MULHERES

In [194]:
df_movies_final_1 = pd.DataFrame()
for index, movie in df_movies_final.iterrows(): #Iterar sobre linhas do DataFrame como pares (índice, série).
    male_count = 0
    female_count = 0
    for producer in movie.producer:
        
        if producer['gender'] == 'male':
            male_count += 1
        else:
            female_count +=1
            
    movie['producer_count_male'] = male_count
    movie['producer_count_female'] = female_count
    
    df_movies_final_1 = pd.concat([df_movies_final_1, pd.DataFrame([movie])], ignore_index=True)

CONTABILIZANDO A QUANTIDADE DE DIRETORES HOMENS E MULHERES

In [195]:
df_movies_final_2 = pd.DataFrame()
for index, movie in df_movies_final_1.iterrows(): #Iterar sobre linhas do DataFrame como pares (índice, série).
    male_count = 0
    female_count = 0
    for director in movie.director:
        
        if director['gender'] == 'male':
            male_count += 1
        else:
            female_count +=1
            
    movie['director_count'] = male_count
    movie['directress_count'] = female_count
    
    df_movies_final_2 = pd.concat([df_movies_final_2, pd.DataFrame([movie])], ignore_index=True)

REMOVE VARIÁVEIS MULTIVALORADAS

In [196]:
df_movies_final_2.drop(['director', 'producer', 'all_actors'], axis= 1 ,inplace= True )
#df_movies_final_2.drop(['all_actors'], axis= 1 ,inplace= True )

CHECA SE AS VARIÁVEIS CRIADAS DE FATO ESTÃO PRESENTES NO DATASET

In [197]:
df_movies_final_2.columns

Index(['title', 'year', 'genre', '_id', 'gender_percent', 'adjusted_budget',
       'actors_count', 'actresses_count', 'producer_count_male',
       'producer_count_female', 'director_count', 'directress_count'],
      dtype='object')

DADOS DE AVALIAÇÕES DO IMDB

In [198]:
df_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1993
1,tt0000002,5.8,268
2,tt0000003,6.5,1879
3,tt0000004,5.5,177
4,tt0000005,6.2,2662
...,...,...,...
1352356,tt9916730,8.3,10
1352357,tt9916766,7.0,22
1352358,tt9916778,7.2,36
1352359,tt9916840,8.8,6


Renomeando a coluna para ter o mesmo nome da coluna de df_movies_final_2


In [199]:
df_ratings.rename(columns={'tconst': '_id'}, inplace=True)

Junção dos dados de averageRating e numVotes a df_movies_final_2


In [200]:
df_movies_final_2 = df_movies_final_2.merge(df_ratings, on='_id')

In [201]:
df_movies_final_2.head()

,title,year,genre,_id,gender_percent,adjusted_budget,actors_count,actresses_count,producer_count_male,producer_count_female,director_count,directress_count,averageRating,numVotes
0,In_Old_Madrid_(1911),1911,"[Comedy, Romance, Short]",tt0001695,50.0,NaN,1,1,1,0,1,0,4.8,62
1,Stick_Around_(1925),1925,"[Comedy, Short]",tt0005864,25.0,NaN,3,1,1,0,1,0,5.5,211
2,Calling_Dr._Porky_(1940),1940,"[Animation, Comedy, Family, Short]",tt0032298,50.0,NaN,1,1,1,0,1,0,6.3,178
3,The_Chewin'_Bruin_(1940),1940,"[Animation, Comedy, Family, Short]",tt0032331,0.0,NaN,2,0,1,0,1,0,6.3,167
4,A_House_Divided_(1913/I),1913,"[Comedy, Short]",tt0002983,50.0,NaN,1,1,0,1,0,1,6.0,532


TRATAMENTO DE DADOS BOX OFFICE

FORMATA NOME DOS FILMES PARA PADRONIZAR DA MESMA FORMA QUE OS NOMES DE FILMES PRESENTES NO DATAFRAME MOVIES

In [202]:
df_box_office['title'] = df_box_office.title.str.replace(' ', '_')

In [203]:
df_box_office.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

REMOVE O ANO DO FINAL DO TITULO DOS FILMES PARA QUE SEJA POSSÍVEL JUNTAR OS DADOS DE AMBOS OS DATAFRAMES PELO TÍTULO POIS O FORMATO DE ID É INCOMPATÍVEL

In [204]:
df_movies_final_2['title'] = df_movies_final_2.title.str.replace("_[\(\[].*?[\)\]]", "",regex=True)

CAPTURANDO ANO EM RELEASE DATE

In [205]:
df_box_office.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates_of_the_Caribbean:_At_World's_End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The_Dark_Knight_Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John_Carter,6.1,2124


In [206]:
df_box_office['release_date'] = pd.to_datetime(df_box_office.release_date)

In [207]:
df_box_office = df_box_office[df_box_office.release_date.notna()]

In [208]:
df_box_office['year'] = df_box_office.release_date.dt.year
df_box_office['year'] = df_box_office.year.astype(int)

C:\Users\950604\AppData\Local\Temp\ipykernel_20900\4014192577.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_box_office['year'] = df_box_office.release_date.dt.year
C:\Users\950604\AppData\Local\Temp\ipykernel_20900\4014192577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_box_office['year'] = df_box_office.year.astype(int)


GARANTE QUE NÃO TERÁ PROBLEMA COM CASE SENSITIVE

In [209]:
df_movies_final_2['title'] = df_movies_final_2.title.str.lower()
df_box_office['title'] = df_box_office.title.str.lower()


C:\Users\950604\AppData\Local\Temp\ipykernel_20900\1218968412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_box_office['title'] = df_box_office.title.str.lower()


JUNTA OS DADOS DE BILHETERIA DE BOX OFFICE EM MOVIES

In [210]:
df_movies_consolidated = df_movies_final_2.merge(df_box_office[['title', 'year', 'budget', 'revenue']], on=['title', 'year'])

In [211]:
df_movies_consolidated = df_movies_consolidated[(df_movies_consolidated.budget > 0) & (df_movies_consolidated.revenue > 0)]

In [212]:
df_movies_consolidated.columns

Index(['title', 'year', 'genre', '_id', 'gender_percent', 'adjusted_budget',
       'actors_count', 'actresses_count', 'producer_count_male',
       'producer_count_female', 'director_count', 'directress_count',
       'averageRating', 'numVotes', 'budget', 'revenue'],
      dtype='object')

In [213]:
df_movies_consolidated.shape

(1942, 16)

CRIA VARIAVEIS DE PORCENTAGEM COM RECORTE EM GENERO SOCIAL PARA ATORES, DIRETORES E PRODUTORES

In [214]:
df_movies_consolidated['actress_percent'] = df_movies_consolidated.actresses_count/(df_movies_consolidated.actors_count + df_movies_consolidated.actresses_count) * 100
df_movies_consolidated['actress_percent'] = df_movies_consolidated['actress_percent'].apply(lambda x: round(x, 0))
df_movies_consolidated['delta_percent'] = df_movies_consolidated.gender_percent - df_movies_consolidated.actress_percent

In [215]:
df_movies_consolidated['actors_percent'] = df_movies_consolidated.actors_count/(df_movies_consolidated.actors_count + df_movies_consolidated.actresses_count) * 100
df_movies_consolidated['actors_percent'] = df_movies_consolidated['actors_percent'].apply(lambda x: round(x, 0))

In [216]:
df_movies_consolidated['women_directing_percent'] = df_movies_consolidated.directress_count/(df_movies_consolidated.director_count + df_movies_consolidated.directress_count) * 100
df_movies_consolidated['women_directing_percent'] = df_movies_consolidated['women_directing_percent'].apply(lambda x: round(x, 0))

In [217]:
df_movies_consolidated['men_directing_percent'] = df_movies_consolidated.director_count/(df_movies_consolidated.director_count + df_movies_consolidated.directress_count) * 100
df_movies_consolidated['men_directing_percent'] = df_movies_consolidated['men_directing_percent'].apply(lambda x: round(x, 0))

In [218]:
df_movies_consolidated['women_producing_percent'] = df_movies_consolidated.producer_count_female/(df_movies_consolidated.producer_count_male + df_movies_consolidated.producer_count_female) * 100
df_movies_consolidated['women_producing_percent'] = df_movies_consolidated['women_producing_percent'].apply(lambda x: round(x, 0))

In [219]:
df_movies_consolidated['men_producing_percent'] = df_movies_consolidated.producer_count_male/(df_movies_consolidated.producer_count_male + df_movies_consolidated.producer_count_female) * 100
df_movies_consolidated['men_producing_percent'] = df_movies_consolidated['men_producing_percent'].apply(lambda x: round(x, 0))

In [220]:
df_movies_consolidated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1942 entries, 0 to 2481
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    1942 non-null   object 
 1   year                     1942 non-null   int64  
 2   genre                    1942 non-null   object 
 3   _id                      1942 non-null   object 
 4   gender_percent           1942 non-null   float64
 5   adjusted_budget          1854 non-null   float64
 6   actors_count             1942 non-null   int64  
 7   actresses_count          1942 non-null   int64  
 8   producer_count_male      1942 non-null   int64  
 9   producer_count_female    1942 non-null   int64  
 10  director_count           1942 non-null   int64  
 11  directress_count         1942 non-null   int64  
 12  averageRating            1942 non-null   float64
 13  numVotes                 1942 non-null   int64  
 14  budget                   1942

COMPARA SE GENDER_PERCENT É UMA VARIAVEL A SER USADA NO LUGAR DE ACTRESS_PERCENT

In [221]:
df_movies_consolidated[df_movies_consolidated.delta_percent != 0]

,title,year,genre,_id,gender_percent,adjusted_budget,actors_count,actresses_count,producer_count_male,producer_count_female,director_count,directress_count,averageRating,numVotes,budget,revenue,actress_percent,delta_percent,actors_percent,women_directing_percent,men_directing_percent,women_producing_percent,men_producing_percent
11,the_texas_chain_saw_massacre,1974,"[Horror, Thriller]",tt0072271,11.0,392169.0,15,2,4,0,1,0,7.4,175269,85000,30859000,12.0,-1.0,88.0,0.0,100.0,0.0,100.0
12,harley_davidson_and_the_marlboro_man,1991,"[Action, Crime, Drama, Thriller, Western]",tt0102005,29.0,39861351.0,26,11,3,1,1,0,6.1,21690,23000000,7434726,30.0,-1.0,70.0,0.0,100.0,25.0,75.0
19,cat_on_a_hot_tin_roof,1958,[Drama],tt0051459,38.0,24000000.0,11,7,1,0,1,0,7.9,52216,3000000,17570324,39.0,-1.0,61.0,0.0,100.0,0.0,100.0
24,topaz,1969,[Thriller],tt0065112,20.0,25316455.0,30,8,2,0,1,0,6.2,19094,4000000,6000000,21.0,-1.0,79.0,0.0,100.0,0.0,100.0
26,the_exorcist,1973,"[Drama, Horror]",tt0070047,36.0,54674187.0,26,15,3,0,1,0,8.1,432016,8000000,441306145,37.0,-1.0,63.0,0.0,100.0,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,old_dogs,2009,"[Comedy, Family]",tt0976238,37.0,38845726.0,63,38,6,1,1,0,5.3,39077,35000000,92219310,38.0,-1.0,62.0,0.0,100.0,14.0,86.0
2476,yes_man,2008,"[Comedy, Romance]",tt1068680,37.0,77177508.0,84,51,5,4,1,0,6.8,375663,70000000,225990978,38.0,-1.0,62.0,0.0,100.0,44.0,56.0
2479,red,2010,"[Action, Comedy, Thriller]",tt1245526,26.0,63112078.0,66,24,6,0,1,0,7.0,318541,58000000,71664962,27.0,-1.0,73.0,0.0,100.0,0.0,100.0
2480,halloween_ii,2009,[Horror],tt1311067,28.0,16648168.0,57,23,10,1,1,0,4.8,58017,15000000,39421467,29.0,-1.0,71.0,0.0,100.0,9.0,91.0


In [222]:
df_movies_consolidated.drop(['_id', 'adjusted_budget', 'delta_percent'], axis= 1 ,inplace= True )


In [223]:
df_movies_consolidated

,title,year,genre,gender_percent,actors_count,actresses_count,producer_count_male,producer_count_female,director_count,directress_count,averageRating,numVotes,budget,revenue,actress_percent,actors_percent,women_directing_percent,men_directing_percent,women_producing_percent,men_producing_percent
0,snow_white_and_the_seven_dwarfs,1937,"[Animation, Family, Fantasy, Musical, Romance]",21.0,11,3,1,0,6,0,7.6,209885,1488423,184925486,21.0,79.0,0.0,100.0,0.0,100.0
1,fantasia,1940,"[Animation, Family, Fantasy, Music]",14.0,6,1,2,0,11,0,7.7,101215,2280000,83320000,14.0,86.0,0.0,100.0,0.0,100.0
2,pinocchio,1940,"[Adventure, Animation, Drama, Family, Fantasy,...",25.0,9,3,1,0,7,0,7.5,155271,2600000,84300000,25.0,75.0,0.0,100.0,0.0,100.0
9,alone_with_her,2006,"[Crime, Drama, Thriller]",58.0,5,7,2,0,1,0,6.0,4164,1000000,10018,58.0,42.0,0.0,100.0,0.0,100.0
10,an_inconvenient_truth,2006,[Documentary],0.0,4,0,7,4,1,0,7.4,84631,1000000,46243000,0.0,100.0,0.0,100.0,36.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,the_wrestler,2008,"[Drama, Romance, Sport]",19.0,93,22,7,2,1,0,7.9,315351,6000000,44703995,19.0,81.0,0.0,100.0,22.0,78.0
2478,i_hope_they_serve_beer_in_hell,2009,[Comedy],44.0,50,40,13,2,1,0,5.2,8637,7000000,1429299,44.0,56.0,0.0,100.0,13.0,87.0
2479,red,2010,"[Action, Comedy, Thriller]",26.0,66,24,6,0,1,0,7.0,318541,58000000,71664962,27.0,73.0,0.0,100.0,0.0,100.0
2480,halloween_ii,2009,[Horror],28.0,57,23,10,1,1,0,4.8,58017,15000000,39421467,29.0,71.0,0.0,100.0,9.0,91.0


CRIA VARIAVEIS DUMMIES PARA GÊNEROS DE FILMES

In [224]:
df = df_movies_consolidated.explode('genre', ignore_index=True)

In [225]:
generos = df.genre.unique()

In [226]:
generos

array(['Animation', 'Family', 'Fantasy', 'Musical', 'Romance', 'Music',
       'Adventure', 'Drama', 'Crime', 'Thriller', 'Documentary', 'Horror',
       'Action', 'Western', 'Comedy', 'War', 'Sci-Fi', 'Sport', 'History',
       'Biography', 'Mystery', 'Film-Noir', 'Adult'], dtype=object)

In [227]:
df_final = pd.DataFrame()
for index, row in df_movies_consolidated.iterrows():
    for genero in generos:
        row[genero] = 1 if genero in row.genre else 0
    df_final = pd.concat([df_final, pd.DataFrame([row])])

In [228]:
df_final.drop('genre', axis=1, inplace=True)

In [229]:
df_final

,title,year,gender_percent,actors_count,actresses_count,producer_count_male,producer_count_female,director_count,directress_count,averageRating,numVotes,budget,revenue,actress_percent,actors_percent,women_directing_percent,men_directing_percent,women_producing_percent,men_producing_percent,Animation,Family,Fantasy,Musical,Romance,Music,Adventure,Drama,Crime,Thriller,Documentary,Horror,Action,Western,Comedy,War,Sci-Fi,Sport,History,Biography,Mystery,Film-Noir,Adult
0,snow_white_and_the_seven_dwarfs,1937,21.0,11,3,1,0,6,0,7.6,209885,1488423,184925486,21.0,79.0,0.0,100.0,0.0,100.0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fantasia,1940,14.0,6,1,2,0,11,0,7.7,101215,2280000,83320000,14.0,86.0,0.0,100.0,0.0,100.0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,pinocchio,1940,25.0,9,3,1,0,7,0,7.5,155271,2600000,84300000,25.0,75.0,0.0,100.0,0.0,100.0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,alone_with_her,2006,58.0,5,7,2,0,1,0,6.0,4164,1000000,10018,58.0,42.0,0.0,100.0,0.0,100.0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,an_inconvenient_truth,2006,0.0,4,0,7,4,1,0,7.4,84631,1000000,46243000,0.0,100.0,0.0,100.0,36.0,64.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,the_wrestler,2008,19.0,93,22,7,2,1,0,7.9,315351,6000000,44703995,19.0,81.0,0.0,100.0,22.0,78.0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2478,i_hope_they_serve_beer_in_hell,2009,44.0,50,40,13,2,1,0,5.2,8637,7000000,1429299,44.0,56.0,0.0,100.0,13.0,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2479,red,2010,26.0,66,24,6,0,1,0,7.0,318541,58000000,71664962,27.0,73.0,0.0,100.0,0.0,100.0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
2480,halloween_ii,2009,28.0,57,23,10,1,1,0,4.8,58017,15000000,39421467,29.0,71.0,0.0,100.0,9.0,91.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [230]:
df_final.to_json('df_movies_consolidated.json')